# Dummy Data

In [1]:
import pandas as pd
import numpy as np
import random
import os

In [2]:
data_path = 'P:\\MyWork\\demoHeaderPredictor\\data'
data_path_external = 'P:\\MyWork\\demoHeaderPredictor\\data\\external'
data_path_model = 'P:\\MyWork\\demoHeaderPredictor\\data\\model'

## Random Geo Coords

In [3]:
def rand_coord():
    return random.uniform(-180,180), random.uniform(-90, 90)

points = (rand_coord() for x in range(2))
for point in points:
    print(point)
    
# other systems: https://gis.stackexchange.com/questions/78838/converting-projected-coordinates-to-lat-lon-using-python

(-71.21191399349453, -25.538663698021836)
(-50.82038438094108, -13.274241458187177)


## Random Emails

In [4]:
import string
domains = [ "hotmail.com", "gmail.com", "aol.com", "mail.com" , "mail.kz", "yahoo.com"]
letters = string.ascii_lowercase[:26]

def get_random_domain(domains):
    return random.choice(domains)

def get_random_name(letters, length):
    return ''.join(random.choice(letters) for i in range(length))

def generate_random_emails(num):
    return [get_random_name(letters, int(random.uniform(2,10))) + '@' + get_random_domain(domains) for i in range(num)]

generate_random_emails(5)

['ke@gmail.com',
 'uvuj@aol.com',
 'yo@yahoo.com',
 'zoacw@hotmail.com',
 'yekhmw@gmail.com']

## Address Data
- contains coords too

In [5]:
import json
try: 
    with open(os.path.join(data_path_external,'address.json')) as json_file:  
        data = json.load(json_file)
    print('File loaded')
except:
    print('Loading failed, curl from git...')
    !curl -o os.path.join(data_path_external,'address.json') "https://raw.githubusercontent.com/EthanRBrown/rrad/master/addresses-us-all.json"
    with open(os.path.join(data_path_external,'address.json')) as json_file:  
            data = json.load(json_file)

File loaded


In [6]:
addresses = pd.concat([pd.DataFrame.from_dict(i,orient='index').T for i in data['addresses']],sort=False)
addresses.reset_index(drop=True,inplace=True)
addresses['lat'] = [x['lat'] for x in addresses['coordinates']]
addresses['lng'] = [x['lng'] for x in addresses['coordinates']]
addresses.drop('coordinates',axis=1, inplace=True)
addresses.rename(columns={'postalCode':'zip'},inplace=True)
addresses[:5]

,address1,address2,city,state,zip,lat,lng
0,1745 T Street Southeast,,Washington,DC,20020,38.867033,-76.979235
1,6007 Applegate Lane,,Louisville,KY,40219,38.134301,-85.649851
2,560 Penstock Drive,,Grass Valley,CA,95945,39.213076,-121.077583
3,150 Carter Street,,Manchester,CT,06040,41.765560,-72.473091
4,2721 Lindsay Avenue,,Louisville,KY,40206,38.263793,-85.700243


## Random Currency

In [7]:
currency = [ "$", "£",'€', "GBP", "USD" , "EUR",]

def get_random_currency(currency):
    return random.choice(currency)

def generate_random_money(num):
    return ['{} {:.3f}'.format(get_random_currency(currency), random.uniform(0,10_000_000)) for i in range(num)]

generate_random_money(5)

['£ 7873445.710',
 '£ 8364827.800',
 '$ 2016084.661',
 '£ 8647339.615',
 'GBP 2277228.228']

## Random Policy References

In [8]:
policy = [ "B", "Z",'S', "V", "W" ]

def get_random_policy(policy):
    return random.choice(policy)

def generate_random_policy(num):
    return ['{}{}{:.0f}'.format(get_random_policy(policy),get_random_name(letters, int(random.uniform(3,4))).upper(), random.uniform(100,999)) for i in range(num)]

generate_random_policy(4)

['SPWH959', 'SGJH549', 'WNCF399', 'BUAG157']

## Random Name & Numbers

In [9]:
try: 
    name_numbers = pd.read_csv(os.path.join(data_path_external,'name_number.txt')) 
    print('File loaded')
except:
    print('Loading failed, curl from git...')
    !curl -o os.path.join(data_path_external,name_number.txt) "https://raw.githubusercontent.com/emilybache/Phone-Numbers-Kata/master/phone_data_10000.txt"
    name_numbers = pd.read_csv(os.path.join(data_path_external,'name_number.txt'))

File loaded


In [10]:
name_split = name_numbers['Name'].str.split(' ').apply(pd.Series)
name_numbers = name_numbers.merge(name_split, how='left', left_index=True,right_index=True).copy()
name_numbers.rename(columns={0:'first_name',1:'last_name', 'Name':'person_name','Phone Number':'phone'}, inplace=True)

# Compose

In [11]:
data = addresses.copy()
data['money'] = generate_random_money(len(data))
data['reference'] = generate_random_policy(len(data))
data['email'] = generate_random_emails(len(data))
data = data.merge(name_numbers[:len(data)], how='left', left_index=True,right_index=True).copy()
data['txt_cat'] = np.random.choice(['A','B','C','D','E'], len(data))
data['num_cat'] = np.random.choice(['0','1','2','3','4'], len(data))
data['numeric'] = [round(random.uniform(1000,9999)) for i in range(len(data))]

In [12]:
data[:5]

,address1,address2,city,state,zip,lat,lng,money,reference,email,person_name,phone,first_name,last_name,txt_cat,num_cat,numeric
0,1745 T Street Southeast,,Washington,DC,20020,38.867033,-76.979235,GBP 487760.350,SPCH308,mex@yahoo.com,Kimberlee Turlington,0345 42 0274,Kimberlee,Turlington,B,3,5181
1,6007 Applegate Lane,,Louisville,KY,40219,38.134301,-85.649851,€ 7321963.108,VDEY870,rg@aol.com,Miguel Eveland,095086-173-31-37,Miguel,Eveland,A,4,2163
2,560 Penstock Drive,,Grass Valley,CA,95945,39.213076,-121.077583,EUR 3341992.053,ZFPH671,ejbyy@hotmail.com,Alonzo Schroyer,057843 018 15-85,Alonzo,Schroyer,D,4,8193
3,150 Carter Street,,Manchester,CT,06040,41.765560,-72.473091,€ 4397323.917,WMDG542,tfanfw@gmail.com,Devon Osei,0698-1368378,Devon,Osei,C,1,6134
4,2721 Lindsay Avenue,,Louisville,KY,40206,38.263793,-85.700243,$ 7755233.425,SLLE128,xm@hotmail.com,Val Hoffmeyer,007 916 73 79,Val,Hoffmeyer,E,0,6663


## Train-Test split

In [18]:
training_data = data[:2000].copy()
testing_data = data[2000:].reset_index(drop=True).copy()
print('Train {} : Test {}'.format(len(training_data),len(testing_data)))

Train 2000 : Test 1220


In [14]:
training_data.to_csv(os.path.join(data_path_model,'training_data.csv'),index=False)

## Prep Test Data

In [19]:
#testing_data.columns = [l for l in letters[:len(data.columns)]]

size=50
list_of_test_dfs = [testing_data.loc[i:i+size-1,:] for i in range(0, len(testing_data),size)]

for e,test_df in enumerate(list_of_test_dfs):
    test_df.to_csv(os.path.join(data_path_model,'testing_data_{}.csv'.format(e)),index=False)